In [1]:
library(compositions)
library(cowplot)
library(ggplot2)
library(reshape2)
library(stringr)
library(lme4)
library(lmerTest)

Loading required package: tensorA

Attaching package: ‘tensorA’

The following object is masked from ‘package:base’:

    norm

Loading required package: robustbase
Loading required package: energy
Loading required package: bayesm
Welcome to compositions, a package for compositional data analysis.
Find an intro with "? compositions"


Attaching package: ‘compositions’

The following objects are masked from ‘package:stats’:

    cor, cov, dist, var

The following objects are masked from ‘package:base’:

    %*%, scale, scale.default

Loading required package: ggplot2

Attaching package: ‘cowplot’

The following object is masked from ‘package:ggplot2’:

    ggsave

Loading required package: Matrix

Attaching package: ‘lmerTest’

The following object is masked from ‘package:lme4’:

    lmer

The following object is masked from ‘package:robustbase’:

    carrots

The following object is masked from ‘package:stats’:

    step



In [2]:
#Load in Data
asv_tab <- read.table("data/asv_table.tsv", sep="\t", comment.char="", skip=1, header=1, row.names=1)
md <- read.table("data/METADATA.txt", sep="\t", header=1, row.names=1)
rownames(md) <- gsub("-",".",rownames(md))
taxonomy <- read.table("data/taxonomy.tsv", sep="\t", header=1, row.names=1)

#Transform data
asv_tab.clr <- t(as.data.frame(clr(t(asv_tab))))
rownames(asv_tab.clr) <- rownames(asv_tab)
asv_tab.clr.m <- melt(asv_tab.clr)
colnames(asv_tab.clr.m) <- c("asv_id","sample_id","clr_rel_abund")
#Merge taxonomic classifications
asv_tab.clr.m$full_taxonomy <- taxonomy[as.vector(asv_tab.clr.m$asv_id), 'Taxon']
asv_tab.clr.m$genus <- str_extract(asv_tab.clr.m$full_taxonomy
                                   ,".*; s__")
asv_tab.clr.m$genus <- str_replace(asv_tab.clr.m$genus,
                                  "; s__", "")
asv_tab.clr.m$family <- str_extract(asv_tab.clr.m$full_taxonomy
                                   ,".*; g__")
asv_tab.clr.m$fmily <- str_replace(asv_tab.clr.m$family,
                                  "; g__", "")
asv_tab.clr.m$phylum <- str_extract(asv_tab.clr.m$full_taxonomy
                                   ,".*; c__")
asv_tab.clr.m$phylum <- str_replace(asv_tab.clr.m$phylum,
                                  "; c__", "")


In [3]:
aggregate_by_level <- function(tax_level) { # level: asv_id, full_taxonomy, genus, family, phylum
    asv_tab.clr.agg <- aggregate(asv_tab.clr.m$clr_rel_abund, 
                                 by=list(asv_tab.clr.m$sample_id,
                                         asv_tab.clr.m[,tax_level]), 
                                 FUN=sum)
    colnames(asv_tab.clr.agg) <- c("sample_id",tax_level,"clr_rel_abund")
    asv_tab.clr.agg$phase <- md[as.vector(asv_tab.clr.agg$sample_id),'phase']
    asv_tab.clr.agg$time <- md[as.vector(asv_tab.clr.agg$sample_id),'timePoint']
    asv_tab.clr.agg$type <- md[as.vector(asv_tab.clr.agg$sample_id),'sampleType']
    asv_tab.clr.agg$subject <- md[as.vector(asv_tab.clr.agg$sample_id),'subject']
    levels(asv_tab.clr.agg$time)[levels(asv_tab.clr.agg$time) == "B"] <- "-7"
    asv_tab.clr.agg$time <- factor(asv_tab.clr.agg$time, levels=c("-7","0","7","14","21","28","35"))
    asv_tab.clr.agg
}

In [4]:
run_GLMs <- function(agg_tab, tax_level) {
    results <- data.frame(row.names = unique(agg_tab[,tax_level]))
    results$SubG_GLM_p <- NA
    results$SupG_GLM_p <- NA
    results$Sal_GLM_p <- NA
    results$T_GLM_p <- NA
    i<-1
    for (tax_str in unique(agg_tab[,tax_level])) {
        for (sample_type in c("SubG", "SupG", "Sal", "T")) {
            data_subset <-agg_tab[agg_tab[,tax_level]==tax_str,]
            data_subset <- subset(data_subset,phase=='induction')
            data_subset <- subset(data_subset,type==sample_type)
            data_subset$time <- as.numeric(as.character(data_subset$time))
            options(warn=-1)
            p.val <- tryCatch({
                # **** THIS IS THE MODEL IF YOU"RE LOOKING FOR IT ****
                fit1 <- lmer(clr_rel_abund~time+(1|subject), data=data_subset)
                p.val <- anova(fit1, type=1)['time',"Pr(>F)"]
                p.val}, 
                error = function(e) {p.val <- NA}, 
                finally = function(e) p.val,
                silent=TRUE)
            options(warn=0)
            results[tax_str, paste(sample_type, "_GLM_p", sep="")] <- p.val
        }
    }
    results[,"SubG_GLM_p_adj"] <- p.adjust(results[,"SubG_GLM_p"], method = "fdr")
    results[,"SupG_GLM_p_adj"] <- p.adjust(results[,"SupG_GLM_p"], method = "fdr")
    results[,"Sal_GLM_p_adj"] <- p.adjust(results[,"Sal_GLM_p"], method = "fdr")
    results[,"T_GLM_p_adj"] <- p.adjust(results[,"T_GLM_p"], method = "fdr")
    results
}

In [5]:
len_wrapper <- function(x, ...) 
{
  paste(strwrap(x, ...), collapse = "\n")
}
cut_taxonomy_string <- function(x, n) {
    y<-str_split(str_trim(str_flatten(str_split_fixed(str_replace_all(x,"[kpcofgs]__", ""), ";", n=7))), " ")[[1]]
    if (n >= length(y)) {
        n = length(y)
    }
    if (n==7) {
        str_flatten(y[c(n-1, n)], collapse = " ")
    } else {
        y[n]
    }
}
plot_significant_results <- function(agg_tab, p_vals, tax_level) {
    plot_list = list()
    i=1
    for (tax_str in rownames(p_vals)) {
        for (sample_type in c("SubG", "SupG", "Sal", "T")) {
            p.val <- p_vals[tax_str,paste(sample_type,"_GLM_p_adj",sep="")]
            if ((!is.na(p.val)) && (p.val <= 0.05)) {
               plot_subset <- agg_tab[agg_tab[,tax_level]==tax_str,]
               plot_subset <- subset(plot_subset, type==sample_type)
               p <- ggplot(plot_subset,
                     aes(x=time,y=clr_rel_abund,group=type,shape=type,col=phase)) +
                     scale_shape_manual(drop=FALSE, values=c(0,1,5,6)) +
                     stat_summary(fun.y=mean, geom="line", aes(group=interaction(1,type)))  + 
                     stat_summary(fun.y=mean, geom="point", size=4, alpha=0.7) +
                     stat_summary(fun.data = mean_se, geom = "errorbar", width=0.1) +
                     scale_colour_brewer(palette = "Dark2") +
                     ggtitle(len_wrapper(cut_taxonomy_string(tax_str,7), 15)) + xlab("") + ylab("") +
                     theme(legend.position="none", 
                           axis.title.y=element_text(size=10),
                           axis.title.x=element_text(size=10),
                           plot.title=element_text(size=10))+
                     annotate("text",label=paste("Adj. p=", round(p.val,3), sep=""),
                              x=-Inf,y=Inf,hjust=-0.1, vjust=1.2, size=3)
               plot_list[[i]] <- p
               i<-i+1
            }
        }
    }
    p.legend <- get_legend(plot_list[[1]] + 
                       theme(legend.position="bottom") + 
                       guides(shape = guide_legend(nrow = 1, title.position="top", title="Site"), 
                              col = guide_legend(nrow = 1, title.position="top", title="Phase")))
    plot_grid(
        plot_grid(plotlist=plot_list, ncol = 4),
        p.legend,
        nrow=2,
        rel_heights=c(1,0.3))
}

In [15]:
agg_tab <- aggregate_by_level("phylum")
p_vals <- run_GLMs(agg_tab, "phylum")

In [16]:
options(repr.plot.width=10, repr.plot.height=2.5)
pdf("figures/Phylum_Sig_GLM.pdf", height=2.5, width=10)
plot_significant_results(agg_tab, p_vals, "phylum")
dev.off()

png 
  2

In [17]:
agg_tab <- aggregate_by_level("family")
p_vals <- run_GLMs(agg_tab, "family")

In [18]:
options(repr.plot.width=10, repr.plot.height=9)
pdf("figures/Family_Sig_GLM.pdf", height=9, width=10)
plot_significant_results(agg_tab, p_vals, "family")
dev.off()

png 
  2

In [20]:
agg_tab <- aggregate_by_level("genus")
p_vals <- run_GLMs(agg_tab, "genus")

In [21]:
options(repr.plot.width=10, repr.plot.height=9)
pdf("figures/Genus_Sig_GLM.pdf", width=10, height=9)
plot_significant_results(agg_tab, p_vals, "genus")
dev.off()

png 
  2

In [22]:
agg_tab <- aggregate_by_level("full_taxonomy")
p_vals <- run_GLMs(agg_tab, "full_taxonomy")

In [23]:
options(repr.plot.width=10, repr.plot.height=10)
pdf("figures/Full_Tax_Sig_GLM.pdf", width=10, height=10)
plot_significant_results(agg_tab, p_vals, "full_taxonomy")
dev.off()

png 
  2

In [26]:
sessionInfo()

R version 3.4.3 (2017-11-30)
Platform: x86_64-pc-linux-gnu (64-bit)
Running under: Ubuntu 18.04.1 LTS

Matrix products: default
BLAS: /usr/lib/x86_64-linux-gnu/blas/libblas.so.3.7.1
LAPACK: /usr/lib/x86_64-linux-gnu/lapack/liblapack.so.3.7.1

locale:
 [1] LC_CTYPE=en_CA.UTF-8       LC_NUMERIC=C              
 [3] LC_TIME=en_CA.UTF-8        LC_COLLATE=en_CA.UTF-8    
 [5] LC_MONETARY=en_CA.UTF-8    LC_MESSAGES=en_CA.UTF-8   
 [7] LC_PAPER=en_CA.UTF-8       LC_NAME=C                 
 [9] LC_ADDRESS=C               LC_TELEPHONE=C            
[11] LC_MEASUREMENT=en_CA.UTF-8 LC_IDENTIFICATION=C       

attached base packages:
[1] stats     graphics  grDevices utils     datasets  methods   base     

other attached packages:
 [1] ggtree_1.12.7       lmerTest_3.0-1      lme4_1.1-18-1      
 [4] Matrix_1.2-8        stringr_1.3.1       reshape2_1.4.3     
 [7] cowplot_0.9.3       ggplot2_3.1.0       compositions_1.40-2
[10] bayesm_3.1-0.1      energy_1.7-5        robustbase_0.93-3  
[13] tenso